# WeatherPy

In [ ]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# import the requests library
import requests
# import the API key
from config import weather_api_key

In [ ]:
# create a set of random lat long combos
lats = np.random.uniform(low=-90,high=90, size =1500)
longs = np.random.uniform(-180,180, 1500)
lat_longs= zip(lats, longs)
lat_longs

# ouput is an array of tuples with indicies

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_longs)

#You can only unzip a zipped tuple once before it is removed from the computer's memory. 
#Make sure you unzip the latitudes and longitudes into the coordinates list before moving on.
# Use the print() function to display the latitude and longitude combinations

# uncomment the for loop to visually confirm the coordinate list
#for coordinate in coordinates:
   # print(coordinate[0], coordinate[1])

In [ ]:
# create a list for holding the cities
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city is unique, add to the cities list
    if city not in cities:
        cities.append(city)
        
# print the city count to confirm sufficient quantity
len(cities)

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
# initialized the counters at 1 bc we want to start both at 1
record_count = 1
set_count = 1

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Loop through all the cities in our list.
# for i in range(len(cities)) # THIS WON'T DO BC NEED CITY NAME NOT INDEX IN URL
# create the for loop with the enumerate() method and reference the index and the city in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    # tell program to pause 60 sec bc the API oly allows 60 calls per min for free
    # prevents time out errors
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    # city_url = url + "&q=" + cities[i]  # THIS URL WONT WORK BC NEED CITY NAME
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count 
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        # pass handles all errors and allows to continue
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame with columns in order
city_data_df = pd.DataFrame(city_data, 
                            columns=["City", "Country", "Date", "Lat", "Lng",
                                     "Max Temp", "Humidity", "Cloudiness", "Wind Speed"])
city_data_df.head(10)

In [ ]:
# Create the output file (CSV).
# specify the path - in this case it will save in same place as WeatherPy.ipynb
output_data_file = "cities.csv" 
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
city_data_df.dtypes